In [3]:
!pip install konlpy
!pip install torchtext

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 13.0MB/s 
     |████████████████████████████████| 3.8MB 60.3MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [12]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
import torch.nn as nn
from konlpy.tag import Okt
#주요 참고 PyTorch로 시작하는 딥 러닝 입문, 유원준

In [ ]:
'''
from google.colab import drive
drive.mount('/content/gdrive')
'''

In [5]:
# naver sentiment corpus
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 23.48 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [18]:
train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [19]:
train_df.drop_duplicates(subset=['document'], inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)


In [8]:
class NsmcDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df, tokenizer):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

In [13]:
class grumodel(nn.Module):
    def __init__(self, embed_dim, vocab_size, hidden_dim, num_layers, batch_size, dropout, use_bn):
        super(GRU, self).__init__()
        self.embed_dim = embed_dim
        self.vocab_size = vocab_size 
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.relu = nn.ReLU()
        self.batch_size = batch_size
        self.use_bn = use_bn


        self.norm = nn.BatchNorm1d(self.batch_size)
        self.embed = nn.Embedding(self.vocab_size,self.embed_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.gru = nn.GRU(self.embed_dim, self.hidden_dim, self.num_layers,bidirectional=True)
        self.gru2 = nn.GRU(self.hidden_dim, self.hidden_dim, self.num_layers,bidirectional=True)

        self.mlp1 = nn.Linear(self.hidden_dim*2,self.hidden_dim)
        self.mlp2 = nn.Linear(self.hidden_dim,self.hidden_dim//2)
        self.mlp3 = nn.Linear(self.hidden_dim//2,self.hidden_dim//4)

    def forward(self,x):
        x = self.embed(x)
        x = self.dropout(x)

        x, _ = self.gru(x)
        x, _ = self.gru2(x)
        x = torch.cat((x[0],x[-1]),dim=1)

        x = self.dropout(x)
        x = self.mlp1(x)
        x = self.relu(x)
        x = self.mlp1(x)
        x = self.relu(x)
        x = self.mlp1(x)
        return x


In [21]:
okt=Okt()
for t in train_df.values:
    print(t)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[7164874
 '케이블에서 다시 안해주나? 악마역의 조민기씨 정말 대단했는데... 그리고 천사역 유호정씨가 빠져있네요~~ 아~ 다시 보고싶다~~~'
 1]
[8188982 '머야 김기열 나와서 19금 붕가붕가 하는거임?ㅡㅡ 장난?' 0]
[9081864 '진짜 간만에 영화봤다는 느낌드네요 대박이다 ㅋㅋㅋ' 1]
[4226459 "ㅎㅎ 다시봐도 죽인다 '비공식적으로 살려줘서 고마워요' '천만에요'" 1]
[8414379 '진짜 전개 자체가 충격임.이런 생각이 참 충격받음' 1]
[4044031 '대박 재밌지!!!! 정경호 연기 너무 웃김 ㅋㅋㅋㅋㅋㅋ 이천희도' 1]
[6996955 '잼있었음' 1]
[9542956 '소음공해인데 더 낮은점수 없나... 네아버는 마이너스 점수를 이 기회에 도입해라' 0]
[9762072 '평화는 마지막까지 신애에게 허락되지 않았고 그녀를 집어삼키려는 밀양은 그녀에게 곧 신이 되었다.' 1]
[8845095 '대박 웃다가 숨넘어가는줄알앗어요재밋어서 파일럿까지 찾아봣어요 ㅋ고생하시는건 아타깝지만 넘넘 재밋어욯' 1]
[8097350 '보면서 느끼는건데 당최 뭔 내용인지 모르겠음' 0]
[6297040 '무신 c바알 때문에 개콘못본다 시ㅡㅂ라년' 0]
[8014120 '0점은 없네 아쉽다' 0]
[9705300 '서기때매 보는건데 조연이라서 진짜 별로다' 0]
[9398025 '팀버튼 전 보고 오자마자 봤는데 정말 괜찮은 애니메이션입니다.' 1]
[9284406 '셋탑박스에 공짜이길래 봤는데 재밌었네요.' 1]
[8900145
 '점수가 살짝 낮아 올리려고 10점. 스토리에 있어 군더더기 없고 배우들 연기 훌륭합니다. 오히려 이 영화가 비극이라는 걸 알고 본다면 더 좋은 것 같아요. 종현은 종현대로 두식은 두식대로.. 업보가 얼마나 무서운 것인지도 잘 그려냄.'
 1]
[9824937 '허접함. 그렇게 많이 야하진않음.' 0]
[8930347 '평점이 왜이리 낮아 재밌

KeyboardInterrupt: ignored

In [16]:
print(train_df[:10])

[['id'], ['document'], ['label']]


In [ ]:
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=128, shuffle=True, num_workers=2)

In [ ]:
device = torch.device("cuda")
model = grumodel()
model.to(device)

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 10000
epochs = 20
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()

        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0
        
        itr+=1
    
    torch.save(model.state_dict(), '/content/gdrive/My Drive/GRUmodel/NLP_Bert_model.pt')

In [ ]:
# evaluation
model.eval()

nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)
